In [1]:
# Lets import the libraries
import socket, cv2, pickle, struct, imutils ,threading

In [2]:
def transmit():
    while True:
        if client_socket:
            url = 'http://192.168.221.209:8080/video' #url given on ip webcam application
            vid = cv2.VideoCapture(url)
            while(vid.isOpened()):
                img,frame = vid.read()
                frame = imutils.resize(frame,width=200)
                a = pickle.dumps(frame)
                message = struct.pack("Q",len(a))+a
                client_socket.sendall(message)
            
                cv2.imshow('TO CLIENT',frame)
                key = cv2.waitKey(1) & 0xFF
                if key ==ord('q'):
                    client_socket.close()
                    break
    cv2.destroyAllWindows()

def recieve(data,payload_size):
    while True:
        while len(data) < payload_size:
            packet = server_socket2.recv(4*1024) 
            if not packet: break
            data+=packet
        packed_msg_size = data[:payload_size]
        data = data[payload_size:]
        msg_size = struct.unpack("Q",packed_msg_size)[0]
        while len(data) < msg_size:
            data += server_socket2.recv(4*1024)
        frame_data = data[:msg_size]
        data  = data[msg_size:]
        frame = pickle.loads(frame_data)
        frame = imutils.resize(frame,width=720)
        cv2.imshow("FROM CLIENT",frame)
        key = cv2.waitKey(1) & 0xFF
        if key  == ord('q'):
            break
    server_socket2.close()
    cv2.destroyAllWindows()

In [3]:
# Create Socket
server_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
host_name  = socket.gethostname()
host_ip = socket.gethostbyname(host_name)
print('HOST IP:',host_ip)
port = 9999
socket_address = (host_ip,port)
# Socket Bind
server_socket.bind(socket_address)

HOST IP: 192.168.99.1


In [4]:
# Socket Listen and accept
server_socket.listen(5)
print("LISTENING AT:",socket_address)
client_socket,addr = server_socket.accept()
print('GOT CONNECTION FROM:',addr)

LISTENING AT: ('192.168.99.1', 9999)
GOT CONNECTION FROM: ('192.168.99.1', 64743)


In [5]:
# connecting to linux socket
server_socket2 = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
host_ip2 = '192.168.99.1' #IP Address of Host to be Entered 
port2 = 6666
server_socket2.connect((host_ip2,port2))

In [6]:
x1=threading.Thread(target=transmit)
data = b""
payload_size = struct.calcsize("Q")
x2=threading.Thread(target=recieve , args=(data , payload_size))
x1.start()
x2.start()

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\sharm\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\sharm\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-9166c083c21b>", line 11, in transmit
OSError: [WinError 10038] An operation was attempted on something that is not a socket
